In [ ]:
import numpy as np

# Load the training and validation datasets
train_data = np.load('train_data.npz')
train_images = train_data['images']
train_labels = train_data['labels']

val_data = np.load('val_data.npz')
val_images = val_data['images']
val_labels = val_data['labels']

# Now you can use train_images, train_labels, val_images, and val_labels for model training

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the image size
IMG_SIZE = 224
# Build the CNN model
model = models.Sequential()

# First convolutional layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third convolutional layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flattening the layers
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))  # Adding dropout to prevent overfitting
model.add(layers.Dense(3, activation='softmax'))  # Assuming 3 classes: fire, smoke, no_fire_smoke

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 26, 26, 128)      

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=16, validation_data=(val_images, val_labels),verbose=1)

Epoch 1/10
1620/1620 [==============================] - 400s 246ms/step - loss: 0.4106 - accuracy: 0.8572 - val_loss: 0.3402 - val_accuracy: 0.8787
Epoch 2/10
1620/1620 [==============================] - 388s 240ms/step - loss: 0.2907 - accuracy: 0.8969 - val_loss: 0.3139 - val_accuracy: 0.8907
Epoch 3/10
1620/1620 [==============================] - 385s 238ms/step - loss: 0.2294 - accuracy: 0.9166 - val_loss: 0.3257 - val_accuracy: 0.8981
Epoch 4/10
1620/1620 [==============================] - 387s 239ms/step - loss: 0.1833 - accuracy: 0.9328 - val_loss: 0.3672 - val_accuracy: 0.8966
Epoch 5/10
1620/1620 [==============================] - 382s 236ms/step - loss: 0.1668 - accuracy: 0.9405 - val_loss: 0.3989 - val_accuracy: 0.8974
Epoch 6/10
1620/1620 [==============================] - 381s 235ms/step - loss: 0.1482 - accuracy: 0.9453 - val_loss: 0.5610 - val_accuracy: 0.8969
Epoch 7/10
1620/1620 [==============================] - 380s 234ms/step - loss: 0.1435 - accuracy: 0.9456 - val_

In [ ]:
val_loss, val_acc = model.evaluate(val_images, val_labels, verbose=2)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

203/203 - 17s - loss: 0.6418 - accuracy: 0.8978 - 17s/epoch - 82ms/step
Validation Accuracy: 89.78%


In [ ]:
model.save('fire_smoke_detector.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
model = tf.keras.models.load_model('fire_smoke_detector.h5')

In [11]:
def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = frame.astype('float32') / 255.0
    frame = np.expand_dims(frame, axis=0)
    return frame

In [10]:
import threading

def process_rtsp_stream(rtsp_url, model):
    cap = cv2.VideoCapture(rtsp_url)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        preprocessed_frame = preprocess_frame(frame)
        prediction = model.predict(preprocessed_frame)
        class_index = np.argmax(prediction)
        labels = ["Fire", "Smoke", "No Fire/Smoke"]
        label = labels[class_index]
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow(rtsp_url, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


# List of RTSP streams
rtsp_streams = ["rtsp://stream1", "rtsp://stream2", "rtsp://stream3"]

# Launch a thread for each stream
threads = []
for stream in rtsp_streams:
    thread = threading.Thread(target=process_rtsp_stream, args=(stream, model))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

